In [8]:
import os 
import pandas as pd
import pybedtools as pbt
import subprocess as sp
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge')
pbt.helpers.set_tempdir('/mnt/BioHome/jreyna/tmp/')
pbt.set_bedtools_path('/mnt/BioApps/bedtools/bin/')
bgzip = '/mnt/BioApps/tabix/tabix-0.2.6/bgzip'
tabix = '/mnt/BioApps/tabix/tabix-0.2.6/tabix'
outdir = 'results/refs/washu/'
os.makedirs(outdir, exist_ok=True)
from biolib import washu

## Load the Gencode Coordinates

In [20]:
gencode = pd.read_table('results/refs/gencode/v30/gencode.v30.annotation.grch37.bed', header=None)
gencode.columns = ['chr', 'start', 'end', 'strand', 'type', 'geneid', 'genename']
gene_coords = gencode.loc[gencode.type == 'gene']

### Make the WashU file for gene coordinates

In [21]:
washu_genes = gene_coords.copy()

# setting very basic coordinates for transcript, translation and exon products
washu_genes['transcript_start'] = washu_genes['start']
washu_genes['transcript_stop'] = washu_genes['end']
washu_genes['translation_start'] = washu_genes['start']
washu_genes['translation_stop'] = washu_genes['end']
washu_genes['transcript_id'] = washu_genes['genename']
washu_genes['exon_start'] = washu_genes['start']
washu_genes['exon_stops'] = washu_genes['end']
washu_genes['type'] = 'coding'

# reordering the dataframe for the WashU browser
refcols = ['chr', 'transcript_start', 'transcript_stop',
           'translation_start', 'translation_stop',
           'strand', 'genename', 'transcript_id', 'type',
           'exon_start', 'exon_stops', 'geneid']
washu_genes = washu_genes.loc[:, refcols]

In [22]:
washu_genes

,chr,transcript_start,transcript_stop,translation_start,translation_stop,strand,genename,transcript_id,type,exon_start,exon_stops,geneid
0,chr1,11869,14409,11869,14409,+,DDX11L1,DDX11L1,coding,11869,14409,ENSG00000223972
12,chr1,14404,29570,14404,29570,-,WASH7P,WASH7P,coding,14404,29570,ENSG00000227232
25,chr1,29554,31109,29554,31109,+,MIR1302-2HG,MIR1302-2HG,coding,29554,31109,ENSG00000243485
33,chr1,34554,36081,34554,36081,-,FAM138A,FAM138A,coding,34554,36081,ENSG00000237613
41,chr1,52473,53312,52473,53312,+,OR4G4P,OR4G4P,coding,52473,53312,ENSG00000268020
...,...,...,...,...,...,...,...,...,...,...,...,...
2782597,GL000195.1,137995,141363,137995,141363,+,CU638689.1,CU638689.1,coding,137995,141363,ENSG00000278878
2782611,GL000220.1,153803,154725,153803,154725,-,FP236383.3,FP236383.3,coding,153803,154725,ENSG00000281181
2782614,GL000205.1,24619,28363,24619,28363,+,CU633904.2,CU633904.2,coding,24619,28363,ENSG00000279647
2782625,GL000220.1,153803,154725,153803,154725,-,FP671120.5,FP671120.5,coding,153803,154725,ENSG00000281383


In [23]:
# writing the dataframe to refbed format with an index
washu_fn = os.path.join(outdir, 'gencode.washu.refbed')
washu.dataframe_to_WashU_refbed(washu_genes, washu_fn)

Running: /mnt/BioApps/tabix/tabix-0.2.6/bgzip -f results/refs/washu/gencode.washu.refbed
Created the gzfn: results/refs/washu/gencode.washu.refbed.gz
Running:  /mnt/BioApps/tabix/tabix-0.2.6/tabix results/refs/washu/gencode.washu.refbed.gz
Created the tabix: results/refs/washu/gencode.washu.refbed.gz.tbi
